# Data extraction
## Wikipedia extraction
### narrative

In [ ]:
import wikipedia as wk
p = wk.search('2010s in film')
page = wk.page(p[0])
s = page.content

In [ ]:
import re
reg = re.compile('\n\n\n=== 0-9 ===\n(.*)\n\n\n== See also ==\n',re.DOTALL)
films = reg.findall(s)[0]
films = films.split('\n')

In [ ]:
films = [f for f in films if(not ('===' in f) and len(f)>0)]

In [ ]:
films[0]
reg2 = re.compile('\(.*\)')
titles = []
for f in films:
    rep = reg2.findall(f)
    if(len(rep)>0):
        titles.append(f.replace(rep[0],''))

In [ ]:
reg3 = re.compile('\n\n\n== Plot ==\n(.*?)\n\n\n==',re.DOTALL)
plots = []
for t in titles:
    p = wk.search(t)
    print p
    try:
        page = wk.page(p[0])
    except:
        continue
    cont = page.content
    plot = reg3.findall(cont)
    if(len(plot)>0):
        plots.append((t,plot[0]))
        #print plot[0]
        #print '\n\n\n'
print len(plots)

In [ ]:
import pandas as pd
df = pd.DataFrame(plots)
df.to_csv("narrative2.csv",sep='\t',encoding='utf-8',index=False,header=['title','plot'])

### informative

In [ ]:
import wikipedia as wk
p = wk.search('science')
page = wk.page(p[0])
s = page.content
links = page.links
print len(links)

In [ ]:
reg3 = re.compile('(.*?)\n\n\n==',re.DOTALL)
articles = []
for t in links:
    p = wk.search(t)
    print p
    try:
        page = wk.page(p[0])
    except:
        continue
    cont = page.content
    article = reg3.findall(cont)
    if(len(plot)>0):
        articles.append((t,article[0]))

print len(articles)

In [ ]:
a_p = [a for a in articles if(len(a[1])>1800)]
df = pd.DataFrame(a_p)
df.to_csv("informative.csv",sep='\t',encoding='utf-8',index=False,header=['title','plot'])

## Making the files

In [ ]:
df = pd.read_csv('../data/narrative2.csv',encoding='utf-8',sep='\t')

l =[]
for i in range(len(df)):
    #print len(df.loc[i]['plot'])
    #print '\n\n\n'
    if(len(df.loc[i]['plot'])>1800):
        l.append(i)

In [ ]:
df = pd.read_csv('../data/informative.csv',encoding='utf-8',sep='\t')
#df = df.loc[l]
write_path='../data/informative/'
for i in range(len(df)):
    df_tmp = df.loc[i]
    filename = unicode(df_tmp['title']).replace(' ','')
    filename = filename.replace('/','')
    file_tmp = open(write_path+filename+".txt", "wb")
    print i
    file_tmp.write(df_tmp["plot"].encode('utf-8'))
    file_tmp.close()
    #df_tmp["plot"].to_csv(df.loc[i][0],sep='\t')

## Web crawling with beautiful soup
### argumentative

In [ ]:
from bs4 import BeautifulSoup
import urllib
import time
import random

r = urllib.urlopen('http://millercenter.org/president/speeches').read()
soup = BeautifulSoup(r)

In [ ]:
urls = soup.findAll('a',attrs={'class':'transcript'})
urls = ['http://millercenter.org'+u['href'] for u in urls]

In [ ]:
speeches = []
for i,u in enumerate(urls):
    r = urllib.urlopen(u).read()
    soup = BeautifulSoup(r)
    try:
        s = soup.findAll("div",attrs={'id':'transcript'})
        speeches.append(s[0].find('p').text)
    except:
        print'error'
        continue            
    print i
    #time.sleep(random.randint(1, 2) * .931467298)

In [ ]:
nspeeches = []
for i,s in enumerate(speeches):
    if(len(s)>1800 and len(s)<10000):
        t = urls[i][urls[i].rfind("/")+1:]
        nspeeches.append((t,s))
        
print(len(nspeeches))

### writing the files

In [ ]:
write_path='../data/argumentative/'
for i in range(len(nspeeches)):
    filename = nspeeches[i][0]
    file_tmp = open(write_path+filename+".txt", "wb")
    file_tmp.write(nspeeches[i][1].encode('utf-8'))
    file_tmp.close()